# 필요한 라이브러리 호출

In [ ]:

import pandas as pd
import numpy as np
import datetime
import glob
import os
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# 데이터 가져오기

In [ ]:
############ 전통조명 미터링 데이터 ############
csv_list_td = [
    '20201101_20201130.csv',
    '20201201_20201231.csv',
    '20210101_20210131.csv',
    '20210201_20210228.csv',
    '20210301_20210331.csv',
    '20210401_20210430.csv',
    '20210501_20210531.csv',
    '20210601_20210630.csv',
    '20210701_20210731.csv',
    '20210801_20210831.csv',
    '20210901_20210930.csv',
    '20211001_20211031.csv',
    '20211101_20211130.csv',
    '20211201_20211231.csv',
    '20220101_20220131.csv',
    '20220201_20220228.csv',
    '20220301_20220331.csv',
    '20220401_20220430.csv',
    '20220501_20220531.csv',
    '20220601_20220630.csv',
    '20220701_20220731.csv',
    '20220801_20220831.csv',
    '20220901_20220930.csv',
    '20221001_20221031.csv',
    '20221101_20221130.csv',
    '20221201_20221231.csv',
]

df_td = pd.DataFrame()
for n in csv_list_td:
    df_td_tmp = pd.read_csv('D:/주거_조명/2020_2021_2022/' + n)
    df_td = pd.concat([df_td, df_td_tmp])

# 미터링게이트웨이 Report 데이터에 포함된 세대수 가져오기

In [ ]:
#대표세대수 가져옴
all_data=pd.read_csv('D:/주거조명_미터링/어울림하트아파트_대표세대수.csv',encoding='Euc-KR')
all_data=all_data.drop(['Unnamed: 0'],axis=1)
#데이터 db 콜럼명 미터링 GID로 맞추기
df_td=df_td.rename(columns={'GID':'미터링_GID'})
#DB에서 대표 mgwgid선정한 mgwgid만 가져오기
result=pd.merge(df_td,all_data,how='outer',indicator=True)
result
mgwgid=result.query('_merge=="both"').drop(columns=['_merge'])

# 데이터 결측치 찾기 

In [ ]:
#mgwgid별로 데이터 프레임 만들기
#data_mgwgid번호_해당년도
for i in gid1_set:
    for j in year:
        globals()['data_'+str(i)+'_'+str(j)]=mgwgid.loc[(mgwgid['미터링_GID']==i)&(pd.to_datetime(mgwgid['USE_DATETIME']).dt.year==j)]
        globals()['data_'+str(i)+'_'+str(j)].loc[:,"USE_DATETIME"]=globals()['data_'+str(i)+'_'+str(j)].loc[:,'USE_DATETIME'].values.astype('<M8[m]')
        globals()['data_'+str(i)+'_'+str(j)]=globals()['data_'+str(i)+'_'+str(j)].reset_index(drop=True)
        globals()['data_'+str(i)+'_'+str(j)]= globals()['data_'+str(i)+'_'+str(j)].drop(['ACTIVE_CURRENT','ACTIVE_VOLTAGE','PF','CRTR_YN','상태_GID','건물 ID','설치_유무'],axis=1)

        
#mgwgid별로 연도별로 나눈 누락된 시간 데이터프레임 만들기
#누락시간대 날짜와 시간을 열로 만들어서 추가
#miss_mgwgid번호_년도
for p in gid1_set:
    for j in year:
        globals()['b_'+str(p)+'_'+str(j)]=pd.date_range((globals()['data_'+str(p)+'_'+str(j)]).loc[0,"USE_DATETIME"],(globals()['data_'+str(p) + '_'+str(j)]).loc[len(globals()['data_'+str(p)+'_'+str(j)])-1,'USE_DATETIME'],freq='T')
        globals()['please_'+str(p)+'_'+str(j)]=pd.DataFrame(globals()['b_'+str(p)+'_'+str(j)])
        globals()['please_'+str(p)+'_'+str(j)].columns=['USE_DATETIME']
        globals()['semi_'+str(p)+'_'+str(j)]=pd.merge(globals()['data_'+str(p)+'_'+str(j)],globals()['please_'+str(p)+'_'+str(j)],how='outer',indicator=True)
        globals()['miss_'+str(p)+'_'+str(j)]=globals()['semi_'+str(p)+'_'+str(j)].query('_merge=="right_only"').drop(columns=['_merge'])

        
#누락시간대 날짜와 연도 시간 분 을 열로 만들어서 추가
for i in gid1_set:
    for j in year:
        globals()['miss_'+str(i)+'_'+str(j)].loc[:,'miss_Year']=pd.to_datetime(globals()['miss_'+str(i)+'_'+str(j)].loc[:,'USE_DATETIME']).dt.year
        globals()['miss_'+str(i)+'_'+str(j)].loc[:,'miss_month']=pd.to_datetime(globals()['miss_'+str(i)+'_'+str(j)].loc[:,'USE_DATETIME']).dt.month
        globals()['miss_'+str(i)+'_'+str(j)].loc[:,'miss_DAY']=pd.to_datetime(globals()['miss_'+str(i)+'_'+str(j)].loc[:,'USE_DATETIME']).dt.day
        globals()['miss_'+str(i)+'_'+str(j)].loc[:,'miss_DAY_hour']=pd.to_datetime(globals()['miss_'+str(i)+'_'+str(j)].loc[:,'USE_DATETIME']).dt.hour
        globals()['miss_'+str(i)+'_'+str(j)].loc[:,'miss_DAY_miniute']=pd.to_datetime(globals()['miss_'+str(i)+'_'+str(j)].loc[:,'USE_DATETIME']).dt.minute
        globals()['miss_'+str(i)+'_'+str(j)]=globals()['miss_'+str(i)+'_'+str(j)].dropna(axis=1)
        
        
#연도별로 나눈 것을 다 합치기
#mgwgid별로 누락된 시간 데이터프레임 만들기

for i in gid1_set:
        globals()['miss_'+str(i)]=pd.concat([globals()['miss_'+str(i)+'_2020'],globals()['miss_'+str(i)+'_2021'],globals()['miss_'+str(i)+'_2022']])
        
        
#연도별로 월별과 일별에 따라 몇분 누락되었는지 분류
#gid_mgwgid번호_miss_group
for i in gid1_set:    
    globals()['gid_'+str(i)+'_miss_group']=(globals()['miss_'+str(i)])['miss_DAY_miniute'].groupby([(globals()['miss_'+str(i)])['miss_Year'],(globals()['miss_'+str(i)])['miss_month'],(globals()['miss_'+str(i)])['miss_DAY']]).count().to_frame()
    
#연도별로 월별과 일별에 따라 몇분 누락되었는지 분류
#gid_mgwgid번호_miss_group
for i in gid1_set:    
    globals()['gid_'+str(i)+'_miss_group']=(globals()['miss_'+str(i)])['miss_DAY_miniute'].groupby([(globals()['miss_'+str(i)])['miss_Year'],(globals()['miss_'+str(i)])['miss_month'],(globals()['miss_'+str(i)])['miss_DAY']]).count().to_frame()  
    
#데이터프레임 너무 많아서 연도별로 나눈거 만듬
#gid_mgwgid번호_해당년도_miss_group
for i in gid1_set:
    for j in year:
        globals()['gid_'+str(i)+'_'+str(j)+'_miss_group']=(globals()['miss_'+str(i)+'_'+str(j)])['miss_DAY_miniute'].groupby([(globals()['miss_'+str(i)+'_'+str(j)])['miss_Year'],(globals()['miss_'+str(i)+'_'+str(j)])['miss_month'],(globals()['miss_'+str(i)+'_'+str(j)])['miss_DAY']]).count().to_frame()    
    

#5분 이상 누락된 데이터만 출력
#gid_mgwgid번호_miss_group_5분이상누락
for i in gid1_set:
    globals()['gid_'+str(i)+'_miss_group_5분이상누락']= (globals()['gid_'+str(i)+'_miss_group'])[(globals()['gid_'+str(i)+'_miss_group'])['miss_DAY_miniute']>5]
    globals()['gid_'+str(i)+'_miss_group_5분이상누락']=globals()['gid_'+str(i)+'_miss_group_5분이상누락'].rename(columns={'miss_DAY_miniute':'total missing time(miniute)'})

#년도까지 포함해서 나누기    
for i in gid1_set:
    for j in year:
        globals()['gid_'+str(i)+'_'+str(j)+'_miss_group_5분이상누락']= (globals()['gid_'+str(i)+'_'+str(j)+'_miss_group'])[(globals()['gid_'+str(i)+'_'+str(j)+'_miss_group'])['miss_DAY_miniute']>5]
        globals()['gid_'+str(i)+'_'+str(j)+'_miss_group_5분이상누락']=globals()['gid_'+str(i)+'_'+str(j)+'_miss_group_5분이상누락'].rename(columns={'miss_DAY_miniute':'total missing time(miniute)'})
        globals()['len_'+str(i)+'_'+str(j)+'_miss_group_5분이상누락']=len(globals()['gid_'+str(i)+'_'+str(j)+'_miss_group_5분이상누락'])